In [ ]:
from clesto import *

#### Parameters

In [ ]:
n = 3
p = 3

arity = p
degree = (n-1)*(p-1)
torsion = p
convention = 'Berger-Fresse'

#### Using the E-infinity construction

In [ ]:
surj = Surjection.steenrod_product(arity, degree, torsion, convention)
print(f'surj = {surj}.\n')
print(f'complexity = {surj.complexity}.\n')
print(f'd_surj = {surj.boundary()}.\n')
print(f'[d_surj] = {surj.boundary().orbit()}')

#### Turchin representative

In [ ]:
x = Surjection_element({(1,2,1):1}, p, convention)
turchin = x.compose(x, 1)
print(f'{x} o_1 {x} = {turchin}')
print(f'its boundary is {turchin.boundary()}')
print(f'whose twisted orbit is {turchin.boundary().orbit("sign")}')

#### All orbits arising from compositions

In [ ]:
from itertools import product 

comp = []
x = Surjection_element({(2,1,2,1): 1}, p, convention)
rho = SymmetricRing.rotation_element(2, torsion=p)

for a, b, i in product([x, rho * x], [x, rho * x], {1,2}):
    comp.append(a.compose(b, i).orbit())
    
for surj in set(comp):
    print(surj)

#### Salvatore representative

In [ ]:
salvatore = Surjection_element({(1, 2, 1, 2, 3, 2, 3):1,
                                (1, 2, 3, 1, 3, 2, 3):1,
                                (1, 2, 3, 2, 1, 2, 3):1,
                                (1, 2, 1, 3, 1, 2, 3):2},
                               torsion, convention)
print(salvatore)
print(salvatore.boundary().orbit())

#### Suspension

In [ ]:
print(salvatore.suspension().boundary().orbit('sign'))

#### Educated guess

In [ ]:
# from itertools import product

# # wild card (1,2,3,2,3,1,3,1,2)

# cand = Surjection_element({(1,2,3,2,1,2,3,2,1):1,(1,2,1,3,1,2,3,2,1):-1,(1,2,3,1,3,2,3,2,1):1,(1,2,1,2,3,2,3,2,1):1,

# (1,2,3,2,1,3,1,2,1):1,(1,2,3,2,3,1,3,2,1):-1,(1,2,3,2,3,2,1,2,1):1},3,'Berger-Fresse')

# keys = [(1,2,1,3,1,3,1,2,1), (1,2,1,3,1,2,1,3,1), (1,2,1,2,1,3,1,3,1), (1,2,1,2,3,2,1,2,1), (1,2,1,3,2,3,1,3,1), (1,2,1,2,3,2,3,1,3), (1,2,1,3,1,3,2,3,2)]
# candidates = []
# for coeff in product([0,1,2], repeat=7):
#     candidate = cand + Surjection_element({k: v for k,v in zip(keys, coeff)}, 3, convention)
#     if not candidate.boundary().orbit('sign'):
#         print(candidate)
# print('finished')

#### All basis up to degree d

In [ ]:
arity = 3
complexity = 3
bases_keys = {}
d = 8
for i in range(0, d+1):
    keys = Surjection.basis(arity, i, complexity)
    # set on a Module_element returns the set of keys
    bases_keys[i] = tuple(set(Surjection_element({k:1 for k in keys}).orbit()))
    print(f'i={i}: {len(bases_keys[i])}\n')

In [ ]:
import scipy.sparse, numpy
def construct_boundary(bases_keys, d, rep='trivial'):
    '''constructs a sparse matrix representing the boundary map from chains of
    degree d to chains of degree d-1'''
    domain_keys, target_keys = bases_keys[d], bases_keys[d-1]
    target_keys_idx = {k: index for index, k in enumerate(target_keys)}
    m, n = len(target_keys), len(domain_keys)
    D = scipy.sparse.lil_matrix((m, n), dtype=int)
    for j, elmt in enumerate(domain_keys):
        bdry = Surjection_element({elmt: 1}).boundary().orbit(rep)
        for k, v in bdry.items():
            D[target_keys_idx[k], j] = v
    return D

boundary = {}
for i in range(1, d+1):
    boundary[i] = (construct_boundary(bases_keys, i, 'sign'))

# boundary square is zero
print((boundary[5] * boundary[6]))

print(boundary[6].todense())

#### From vector to chain

In [ ]:
d = 6
idx2key = {idx: key for idx, key in enumerate(bases_keys[d])}
vector = (0,0,0,0,0,2,0,0,0,2,0,1,0,1,1,0,0)
chain = Surjection_element(torsion=arity)
for idx, v in enumerate(vector):
    if v:
        chain += chain.create({idx2key[idx]: v})
print(chain, '\n')
print(chain.boundary().orbit('sign'))